In [1]:
import tkinter as tk
BOM=50 #board_origin_margin
DD=50 #dots distance
DR=10 #dots radius
class DotAndBoxes:
    def __init__(self, master):
        self.master = master
        self.master.title('Dots and Boxes')
        self.canvas = tk.Canvas(self.master, bg='white', width=600, height=600)
        self.canvas.pack(fill='both', expand=True)
        self.board = Board()
        self.draw_board()
        self.draw_lines()
        self.canvas.bind('<Button-1>', self.on_click)
        self.current_player = 1
        self.player1_score = 0
        self.player2_score = 0
        self.player1_score_label = tk.Label(self.master, text='Player 1: {}'.format(self.player1_score), font=('Arial', 14),
                                            bg="cyan",relief=tk.RAISED)
        self.player1_score_label.pack(side='left', padx=10, pady=10)
        self.player2_score_label = tk.Label(self.master, text='Player 2: {}'.format(self.player2_score), font=('Arial', 14),
                                           relief=tk.RAISED)
        self.player2_score_label.pack(side='right', padx=10, pady=10)

    def draw_board(self):
       
        for i in range(11):
            for j in range(11):
                x1 = i * DD + BOM
                y1 = j * DD + BOM
                x2 = x1 + DR
                y2 = y1 + DR
                if i==0 and j==0:
                    self.canvas.create_oval(x1, y1, x2, y2, fill='red')
                else:
                    self.canvas.create_oval(x1, y1, x2, y2, fill='black')


    def draw_lines(self):
        for line in self.board.lines:
            #+DR/2 : create_oval center shift
            x1 = line[0] * 50 + 50+DR/2
            y1 = line[1] * 50 + 50+DR/2
            x2 = line[2] * 50 + 50+DR/2
            y2 = line[3] * 50 + 50+DR/2
            #player 1 blue
            if line[4] == 1:
                self.canvas.create_line(x1, y1, x2, y2, width=5, fill='cyan')
            #player 2 green
            elif line[4] == 2:
                self.canvas.create_line(x1, y1, x2, y2, width=5, fill='green')

    def on_click(self, event):
        # Get the x and y coordinates of the click event
        x, y = event.x, event.y
        '''
        # Check if the click is close to any of the dots on the board
        for dot in self.board.dots:
            if abs(x - dot[0]) <= 10 and abs(y - dot[1]) <= 10:
                # If the click is close to a dot, check if there are any nearby dots
                for other_dot in self.board.dots:
                    if dot != other_dot and abs(x - other_dot[0]) <= 10 and abs(y - other_dot[1]) <= 10:
                        # If there is another nearby dot, create a new line object between the two dots
                        line = [dot[0], dot[1], other_dot[0], other_dot[1], 0]
                        self.board.lines.append(line)
                        self.draw_lines()
                        self.check_boxes()
                        if self.current_player == 1:
                            self.current_player = 2
                        else:
                            self.current_player = 1
                        self.update_score()
                        self.check_game_over()
                        return
        '''
        # origin of board is like a 2D matrix (0,0) (because we have 2D monitors in common)
        # so (0,0) stands in the upper left point/dot
        for i,line in enumerate(self.board.lines):
            if len(line) < 5:
                continue
            #check line ==0 i.e. not belongs to any player, without tag/free
            if line[4] == 0:
                #get the line coordinate, (x1,y1) as start; (x2, y2) as end.
                x1 = line[0] * DD + BOM+DR/2
                y1 = line[1] * DD + BOM+DR/2
                x2 = line[2] * DD + BOM+DR/2
                y2 = line[3] * DD + BOM+DR/2
                #click boarders of lines
                # we choose a Rhombus i.e. clicks should be between two parallel lines
                # So the chosen line is vertical/horizental axe of the Rhombus
                # we have (y+x = a) parallel lines // or (y-x=a) \\
                # y1==y2 horizental lines (y1==y2) 
                if ((x1+y1<x+y and x+y<x2+y2) and y1==y2):
                    if (y2-x2<y-x and y-x<y1-x1):
                        self.check_lines(i)
                # x1==x2: vertical lines
                if ((x1+y1<x+y and x+y<x2+y2) and x1==x2):
                    if (y1-x1<y-x and y-x<y2-x2):
                        self.check_lines(i)
    def check_lines(self,i):
        #
        line=self.board.lines[i]
        line[4] = self.current_player
        self.draw_lines()
        self.check_boxes()
        self.update_score()
        self.check_game_over()
        if self.current_player == 1:
            self.current_player = 2
            self.player1_score_label.config(bg="SystemButtonFace")
            self.player2_score_label.config(bg="green")
        else:
            self.current_player = 1
            self.player1_score_label.config(bg="cyan")
            self.player2_score_label.config(bg="SystemButtonFace")
                
    def check_boxes(self):
        for box in self.board.boxes:
            if box[4] == 0:
                count = 0
                box_lines=[box[0]+box[1],
                           box[1]+box[2],
                           box[0]+box[3],
                           box[3]+box[2]]
                for line in box_lines:
                    if line+[0] in self.board.lines:
                        break
                    elif line+[self.current_player] in self.board.lines:
                        count += 1
                    else:
                        break
                if count == 4:
                    box[4] = self.current_player
                    if self.current_player == 1:
                        self.player1_score += 1
                    else:
                        self.player2_score += 1

    def update_score(self):
        self.player1_score_label.config(text='Player 1: {}'.format(self.player1_score))
        self.player2_score_label.config(text='Player 2: {}'.format(self.player2_score))

    def check_game_over(self):
        game_over = True
        for line in self.board.lines:
            if line[4] == 0:
                game_over = False
                break
        if game_over:
            self.show_winner()

    def show_winner(self):
        if self.player1_score > self.player2_score:
            winner = 'Player 1'
        elif self.player2_score > self.player1_score:
            winner = 'Player 2'
        else:
            winner = 'Tie'
        msg = tk.messagebox.showinfo('Game Over', 'The winner is {}'.format(winner))
        if msg == 'ok':
            self.master.quit()

class Board:
    def __init__(self):
        self.lines = []
        for i in range(11):#rows,x
            for j in range(11):#columns,y
                if i < 10:#
                    self.lines.append([i, j, i+1, j, 0])
                if j < 10:#
                    self.lines.append([i, j, i, j+1, 0])
        self.boxes = []
        for i in range(10):
            for j in range(10):
                self.boxes.append([[i, j], [i+1, j], [i+1, j+1], [i, j+1], 0])

if __name__ == '__main__':
    root = tk.Tk()
    app = DotAndBoxes(root)
    root.mainloop()